In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
import app.netools as net
nt=net.NetTools()
import matplotlib.pyplot as plt
import app.mydef as md
mt = md.MyTools()
from shapely.geometry import *
import app.main as m

In [4]:
mt.pip('mapbox')

In [109]:
wuhan_tu = gpd.read_file('g:/1-规划/图层/武汉分公司图层20190109/武汉市区分公司区域图层201804版/武汉市区分公司区域图层201804版/武汉市区分公司区域图层201804版.TAB')

In [110]:
wuhan_tu = wuhan_tu.to_crs({'init': 'epsg:4326'}) 

In [57]:
data = pd.read_csv(open(r'G:/1-规划/图层/栅格/2019年8月/0805/0806grid.csv',encoding = 'gbk'))

In [58]:
data['lon'] = (data['a.grid_leftbottom_longitude']+data['a.grid_righttop_longitude'])/2
data['lat'] = (data['a.grid_leftbottom_latitude']+data['a.grid_righttop_latitude'])/2

In [59]:
data_gis = nt.add_points(data,coords=['lon','lat','',''])

In [60]:
data_7734 = pd.read_excel('G:/1-规划/1其他工作输出/20190807数据质检7734/7734经纬度.xlsx',encoding = 'gbk')

In [61]:
data_7734_gis = nt.add_buffer_df(data_7734, lon='RRU经度', lat='RRU纬度', buff_col='距离')

In [104]:
data_res = gpd.sjoin(data_7734_gis,data_gis)

In [105]:
data_res1=data_res.drop_duplicates(subset='a.grid_id',keep='first',inplace=False)

In [107]:
data_res1.columns = ['cgi1', 'RRU经度1', 'RRU纬度1', '场景1', '距离1', 'geometry', 'index_right1',
       'a.region_name', 'a.grid_id', 'a.grid_leftbottom_longitude',
       'a.grid_leftbottom_latitude', 'a.grid_lefttop_longitude',
       'a.grid_lefttop_latitude', 'a.grid_righttop_longitude',
       'a.grid_righttop_latitude', 'a.grid_rightbottom_longitude',
       'a.grid_rightbottom_latitude', 'b.ul_data', 'b.dl_data',
       'a.rsrp_sample_count', 'a.rsrp_sum', 'a.rsrp_weak', 'a.scrsrp_95',
       'a.scrsrp_110_95', 'a.scrsrp_110', 'a.inter_maxrsrp_weak',
       'a.un_maxrsrp_sum', 'a.un_maxrsrp_sample_count', 'a.un_maxrsrp_weak',
       'a.te_maxrsrp_weak', 'a.te_maxrsrp_sample_count', 'a.te_maxrsrp_sum',
       'a.te_maxrsrp_weak_l2', 'a.un_maxrsrp_weak_l2', 'lon', 'lat']

In [111]:
data_res2 = gpd.sjoin(wuhan_tu,data_res1)

In [113]:
data_res3=data_res2.drop_duplicates(subset='a.grid_id',keep='first',inplace=False)

In [115]:
data_res_grid=nt.add_pologons_from_grids(data_res3, 
                                         coords=['a.grid_leftbottom_longitude', 'a.grid_leftbottom_latitude', 'a.grid_lefttop_longitude', 'a.grid_lefttop_latitude', 'a.grid_righttop_longitude', 'a.grid_righttop_latitude', 'a.grid_rightbottom_longitude', 'a.grid_rightbottom_latitude']
                                         , geometry='geometry'
                                         , color_col='color'
                                         , coverage_limits=(0.7, 0.9)
                                         , colors=['ff0000ff', 'ff00ffff', 'ff008000']
                                         , check_sample_counts=False, low_sample_counts=0, low_sample_counts_color='white')

G:\mypy\app\netools.py:589: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[geometry] = df[coords].apply(add_a_polygon_from_a_grid, axis=1)


In [116]:
data_res_grid1 = data_res_grid.merge( data_res, how='inner',on='a.grid_id')

In [117]:
data_res_grid2 = data_res_grid1[['a.region_name_x', 'a.grid_id', 'a.rsrp_sample_count_x',
       'a.rsrp_weak_x', 'coverage', 'color', 'geometry_x','lon', 'lat']]

In [118]:
data_res_grid2.columns = ['a.region_name', 'a.grid_id', 'a.rsrp_sample_count',
       'a.rsrp_weak', 'coverage', 'color', 'geometry','lon', 'lat']

In [119]:
data_res_grid2 = data_res_grid2.set_geometry('geometry')

In [120]:
import simplekml
import pandas as pd
from shapely.geometry import Polygon
from geographiclib.geodesic import Geodesic

In [121]:
#---------找lod的四个角
def _lod_lon_lat(lon=114.198312,lat=30.630697,azimuth=45,azimuth1=225,radius=7000):#^^^^^^^设置视野
    res = Geodesic.WGS84.Direct(lat, lon, azimuth, radius)
    lon_1= res['lon2']
    lat_1= res['lat2']
    res1 = Geodesic.WGS84.Direct(lat, lon, azimuth1, radius)
    lon_2= res1['lon2']
    lat_2= res1['lat2']
    return lon_1,lat_1,lon_2,lat_2

In [122]:
data_res_grid3 = data_res_grid2.drop_duplicates(subset='a.grid_id',keep='first',inplace=False)

In [123]:
row_table = data_res_grid3.copy()

In [124]:
id_n = 'a.grid_id'
lon_n = 'lon'
lat_n = 'lat'
yanse = 'color'
shiye_n = 7000

In [125]:
kml = simplekml.Kml()
lod1 = simplekml.Lod(minlodpixels=788, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
for i in range(len(row_table.index)):
    row_1 = row_table.iloc[i,:]
    id_n_v,polygon1,lon_yuan,lat_yuan,yanse_yuan = row_1[[id_n,'geometry',lon_n,lat_n,yanse]]
    lon_dd,lat_dd,lon1_dd,lat1_dd = _lod_lon_lat(lon=lon_yuan, lat=lat_yuan, radius=shiye_n)
    latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                      minaltitude=None, maxaltitude=None, altitudemode=None)
    
    pol = kml.newpolygon(name=id_n_v)#,description = biaozhu_r
    pol.region.latlonaltbox = latlonaltbox
    pol.region.lod = lod1
    pol.outerboundaryis =polygon1.exterior.coords# polygon_1 是一个扇区
    
    colour = yanse_yuan  #^^设置一个颜色
    colour_alpha = simplekml.Color.changealphaint(100, colour)#^^^^^将颜色添加透明度0-255
    pol.linestyle.color = colour_alpha# 最终线条上色    
    pol.polystyle.color = colour_alpha# 最终形状上色
kml.save("222_扇区图层.kml")

In [126]:
print('ok--------------------------\n-------\n\n-------\n\n-------\n\n-------\n\n-------\n\n-------\n')

ok--------------------------
-------

-------

-------

-------

-------

-------

